In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", message="PerformanceWarning")
warnings.filterwarnings("ignore", message="UserWarning")

# TS
from neuralprophet import NeuralProphet, uncertainty_evaluate
from sktime.forecasting.model_selection import temporal_train_test_split

# User Imports
import sys
sys.path.append("..")
from src import util

%load_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize'] = (14, 7)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# modeling data
load = util.read_load("../data/load_hist_data.csv")
weather = util.read_weather("../data/weather_data.csv")
weather_features = util.featurize_weather(
    weather, lags=[24]
)  # 24 hours = 1 day lagged weather
mod_data = util.create_mod_data(load, weather_features)
mod_data.drop(columns=['school_break'], inplace=True)


inference_data = mod_data[mod_data.ds >= "2008-01-01"]
mod_data = mod_data[mod_data.ds < '2008-01-01'] 

train_data, test_data = temporal_train_test_split(mod_data, test_size=1 / 3)
tune_data, test_data = temporal_train_test_split(test_data, test_size=1 / 3)

In [4]:
m = NeuralProphet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    n_lags=1*24,
    ar_reg=0.5,
    epochs=20,
    n_forecasts = 2*24, # steps ahead to forecast
)
for i in range(7):
    m.add_seasonality(
        name=f"daily_dow{i}",
        period=1,
        fourier_order=4,
        condition_name=f"dow_{i}",
        # prior_scale=dow_prior,
    )
m.add_country_holidays(country_name="US")
m.add_future_regressor("max_station_temp")
m.add_future_regressor("min_station_temp")
m.add_future_regressor("mean_station_temp")
m.add_future_regressor("lag_24__min_station_temp")
m.add_future_regressor("lag_24__max_station_temp")
m.add_future_regressor("lag_24__mean_station_temp")

m.fit(
    df = train_data,
    freq='H',
    validation_df=tune_data
)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.994% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64


Finding best initial lr:   0%|          | 0/256 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

,MAE_val,RMSE_val,Loss_val,RegLoss_val,epoch,MAE,RMSE,Loss,RegLoss
0,2715.265381,3738.573486,1.917470,0.000000,0,3783.059814,4808.628906,2.138438,0.000000
1,537.813904,688.843567,0.173360,0.000000,1,1114.806763,1503.041992,0.455889,0.000000
2,277.624115,354.921783,0.050336,0.000000,2,319.475098,417.197479,0.051665,0.000000
3,228.814438,293.889648,0.033880,0.000000,3,211.491943,277.666046,0.022199,0.000000
4,195.072922,250.577637,0.024373,0.000000,4,181.375122,237.731735,0.016266,0.000000
5,170.854721,219.564331,0.018587,0.000000,5,157.817093,207.123932,0.012344,0.000000
6,157.319412,202.644882,0.015743,0.000000,6,143.032822,188.649261,0.010208,0.000000
7,149.278458,193.104843,0.014255,0.000000,7,134.930878,178.695114,0.009136,0.000000
8,144.551147,187.371048,0.013412,0.000000,8,130.482224,173.145660,0.008557,0.000000
9,141.915512,184.077515,0.012936,0.000000,9,127.535599,169.618805,0.008192,0.000000


## Conformal Prediction

In [ ]:
confidence_lvl = 0.90
quantile_list = [round(((1 - confidence_lvl) / 2), 2), round((confidence_lvl + (1 - confidence_lvl) / 2), 2)]
method = 'naive'
alpha = 1 - confidence_lvl


m_cp = NeuralProphet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    n_lags=1*24,
    ar_reg=0.5,
    epochs=20,
    n_forecasts=2*24, # steps ahead to forecast
    quantiles=quantile_list
)
for i in range(7):
    m_cp.add_seasonality(
        name=f"daily_dow{i}",
        period=1,
        fourier_order=4,
        condition_name=f"dow_{i}",
    )
m_cp.add_country_holidays(country_name="US")
m_cp.add_future_regressor("max_station_temp")
m_cp.add_future_regressor("min_station_temp")
m_cp.add_future_regressor("mean_station_temp")
m_cp.add_future_regressor("lag_24__min_station_temp")
m_cp.add_future_regressor("lag_24__max_station_temp")
m_cp.add_future_regressor("lag_24__mean_station_temp")

train_metrics = m_cp.fit(
    df = train_data,
    freq='H',
)

In [ ]:
tune_fcst = m_cp.predict(tune_data)

In [ ]:
quantile_fcst = tune_fcst[['ds', 'y'] + [cn for cn in tune_fcst.columns if 'yhat48' in cn]].copy()
g=sns.lineplot(data=quantile_fcst, x='ds', y='yhat48', label='forecast')
sns.scatterplot(data=quantile_fcst, x='ds', y='y', color='red', label='actual')
ax = plt.gca()
ax.fill_between(
    quantile_fcst.ds,
    quantile_fcst['yhat48 5.0%'],
    quantile_fcst['yhat48 95.0%'],
    alpha=0.5,
    color='green',
    label=f"{confidence_lvl:.0%} quantile band")
ax.legend()
g.set_title("Neural Prophet Forecast with Quantile Intervals")

In [ ]:

naive_cp_forecast = m_cp.conformal_predict(
    pd.concat([train_data, tune_data]),
    calibration_df=test_data,
    alpha=alpha,
    method=method,
    plotting_backend="plotly-static",
    # show_all_PI=True,
)

In [ ]:

interval_fcst = naive_cp_forecast[['ds', 'y'] + [cn for cn in naive_cp_forecast.columns if 'yhat48' in cn]].copy()
g=sns.lineplot(data=interval_fcst, x='ds', y='yhat48', label='forecast')
sns.scatterplot(data=interval_fcst, x='ds', y='y', color='red', label='actual')
ax = plt.gca()
ax.fill_between(
    interval_fcst.ds,
    interval_fcst['yhat48 5.0%'],
    interval_fcst['yhat48 95.0%'],
    alpha=0.5,
    color='purple',
    label=f"{confidence_lvl:.0%} conformal interval")
ax.legend()
g.set_title("Neural Prophet Forecast with Conformal Prediction Intervals")

In [ ]:
uncertainty_res = uncertainty_evaluate(naive_cp_forecast)
uncertainty_res = uncertainty_res.T.reset_index()
uncertainty_res.columns = ['horizon', 'method', 'metric', 'value']
uncertainty_res['horizon_num'] = uncertainty_res['horizon'].str.slice(4,).astype(int)
sns.relplot(
    data=uncertainty_res[~uncertainty_res.metric.str.contains('qhat')],
    x="horizon_num", y="value",
    hue="method", col="metric",
    kind="line", facet_kws=dict(sharex=True, sharey=False),
)

## Retrain on all Data

In [ ]:
# Retrain with all data - use last 2 months as calibration data

deploy_mod = NeuralProphet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    n_lags=1*12,
    ar_reg=0.5,
    epochs=20,
    n_forecasts=7*24, # steps ahead to forecast
    quantiles=quantile_list
)
for i in range(7):
    deploy_mod.add_seasonality(
        name=f"daily_dow{i}",
        period=1,
        fourier_order=4,
        condition_name=f"dow_{i}",
    )
deploy_mod.add_country_holidays(country_name="US")
deploy_mod.add_future_regressor("max_station_temp")
deploy_mod.add_future_regressor("min_station_temp")
deploy_mod.add_future_regressor("mean_station_temp")
deploy_mod.add_future_regressor("lag_24__min_station_temp")
deploy_mod.add_future_regressor("lag_24__max_station_temp")
deploy_mod.add_future_regressor("lag_24__mean_station_temp")

deploy_metrics = deploy_mod.fit(
    df = mod_data,
    freq='H',
)
deploy_metrics

In [ ]:
inference_preds = deploy_mod.conformal_predict(
    pd.concat([mod_data, inference_data]),
    calibration_df=test_data,
    alpha=alpha,
    method='naive'
)

In [ ]:
for cn in inference_preds.columns:
    print(cn)

inference_preds[['ds', 'y'] + [f"qhat{i}" for i in range(1,49)]]

In [ ]:
deploy_cp_forecast = deploy_mod.conformal_predict(
    inference_data.drop(columns='y'),
    calibration_df=test_data,
    alpha=alpha,
    method=method,
    plotting_backend="plotly-static",
    # show_all_PI=True,
)

In [ ]:
deploy_cp_forecast.tail()

In [ ]:


naive_forecast1 = m_cp.conformal_predict(
    df=tune_data.drop(columns='school_break'),
    calibration_df=test_data.drop(columns='school_break'),
    alpha=alpha,
    method=method,
    plotting_backend='plotly_static'
)

In [ ]:
naive_forecast1.columns
naive_forecast1[['ds', 'y', 'yhat48', 'yhat48 - qhat48', 'yhat48 + qhat48']]

In [ ]:
forecast1 = m.predict(test_data.drop(columns='school_break'))[24:]

In [ ]:
foo = forecast1[forecast1.yhat24.notna()][['ds', 'y', 'yhat24']].copy()
foo['resid'] = foo['y'] - foo['yhat24']

sns.lineplot(data=foo, x='ds', y='resid')

In [ ]:
?NeuralProphet

In [ ]:
?m.plot_components

In [ ]:
?m.get_latest_forecast

In [ ]:
tune_fcst[['ds', 'y'] + [cn for cn in tune_fcst.columns if "yhat" in cn]]

In [ ]:
fcst = m.predict(
    mod_data[[
        'ds', 'y',
        'dow_0', 'dow_1', 'dow_2', 'dow_3', 'dow_4', 'dow_5', 'dow_6',
        'max_station_temp', 'min_station_temp', 'mean_station_temp',
        'lag_24__min_station_temp', 'lag_24__max_station_temp', 'lag_24__mean_station_temp',
    ]]
)
fcst

# m.plot(fcst)

In [ ]:
fcst['resid24'] = fcst['y'] - fcst['yhat24']


In [ ]:
sns.lineplot(data=fcst, x='ds', y='resid24')

In [ ]:
m.get_latest_forecast(fcst)

In [ ]:
m.plot_components(fcst)

In [ ]:
fcst['residual'] = fcst['y'] - fcst['yhat1']

In [ ]:
util.slider_plot(data=fcst, x='ds', y='residual')

In [ ]:
!pip3 install statsmodels

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf
plot_data = fcst[fcst.residual.notna()]
acf(plot_data['residual'])
plot_acf(plot_data['residual'])

In [ ]:
mod_data[:-3].apply(lambda x: x.isna().sum())

In [ ]:
forecast.head()

In [ ]:
m.plot(forecast)

In [ ]:
confidence_lvl = 0.90
quantile_list = [round(((1 - confidence_lvl) / 2), 2), round((confidence_lvl + (1 - confidence_lvl) / 2), 2)]
method = 'naive'
alpha = 1 - confidence_lvl

In [ ]:
inference_preds = pd.read_csv("../data/inference_preds_nforecasts336.csv")
# inference_preds.ds = pd.to_datetime(inference_preds)

In [ ]:
conformal_std = inference_preds[[cn for cn in inference_preds.columns if 'qhat' in cn and 'yhat' not in cn]].apply(max).reset_index()
conformal_std.columns = ['horizon', 'qhat']
conformal_std['horizon'] = conformal_std.horizon.str[4:].astype(np.float64) / 24
sns.lineplot(data=conformal_std, x='horizon', y='qhat')

In [ ]:
conformal_std

In [ ]:
inference_preds[inference_preds.ds >= '2008-01-01']